In [46]:
%load_ext autoreload
%autoreload 2
%pylab inline

import pyspiel
import open_spiel.python.examples.ubc_utils
from open_spiel.python.visualizations import ubc_treeviz
from open_spiel.python.examples.ubc_utils import *
from absl import logging
logging.set_verbosity(logging.INFO)
from open_spiel.python.examples.ubc_nfsp_example import NFSPPolicies, policy_from_checkpoint
from open_spiel.python.examples.ubc_br import checkpoint_sub_agent_i, dqn_agent_from_checkpoint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/apps/open_spiel/venv/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['solve', 'random', 'logging']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [47]:
EXPERIMENT_DIR = '/shared/outputs/jan28_big/mlp'
CHECKPOINT_NAME = 'checkpoint_latest'
### Load game
game = smart_load_sequential_game('clock_auction', game_parameters=dict(filename=f'{EXPERIMENT_DIR}/game.json'))

# CHECKPOINT_NAME = 'checkpoint_1500000'

In [48]:
policy = policy_from_checkpoint(EXPERIMENT_DIR, CHECKPOINT_NAME).nfsp_policies

# Alternatively, to load a BR
# br_agent = dqn_agent_from_checkpoint(EXPERIMENT_DIR, 'checkpoint_3000000', 'checkpoint_3000000_br_0')
# policy = checkpoint_sub_agent_i(EXPERIMENT_DIR, 'checkpoint_3000000', br_agent)

INFO:absl:Game loaded
INFO:absl:Using game instance: turn_based_simultaneous_game
INFO:absl:Game has a state size of 646, 12 distinct actions, and 2 players
INFO:absl:Game has 3 products
INFO:absl:Creating NFSP using device: cuda for player 0
INFO:absl:Double DQN activated for player 0
INFO:absl:Creating DQN using device: cuda for player 0
INFO:absl:Creating NFSP using device: cuda for player 1
INFO:absl:Double DQN activated for player 1
INFO:absl:Creating DQN using device: cuda for player 1


In [15]:
# Alternatively, to get from CFR directly
# solver = pyspiel.CFRSolver(game)
# for i in range(1000):
#     solver.evaluate_and_update_policy()
# policy = solver.average_policy()

In [49]:
node_policy_decorator, edge_policy_decorator = ubc_treeviz.make_policy_decorators(policy)

In [50]:
gametree = ubc_treeviz.GameTree(
    game,
    node_decorator=node_policy_decorator,
    edge_decorator=edge_policy_decorator,
    group_infosets=True,
    group_terminal=False,
    group_pubsets=False, 
    target_pubset='*',
    depth_limit=10,
    state_prob_limit=0.01,
    action_prob_limit=0.05, 
    policy=policy
)

outfile= f'./graphviz/{game_name}.pdf'

# logging.info("Game tree:\n%s", gametree.to_string())

gametree.draw(outfile, prog='dot')
logging.info("Game tree saved to file: %s", outfile)


INFO:absl:Building tree...
INFO:absl:Built tree!
INFO:absl:Game tree saved to file: ./graphviz/big_game_2.pdf
